普通方法

In [2]:
import torch

x = torch.tensor([1,2,3]).float()
A=torch.nn.Linear(3, 5)._parameters['weight']
B=torch.nn.Linear(5, 5)._parameters['weight']
C=torch.nn.Linear(5, 1)._parameters['weight']

x.matmul(A.T).matmul(B.T).matmul(C.T)

tensor([-0.0106], grad_fn=<SqueezeBackward4>)

权重运算符重载

In [152]:
class NetMat():
    def __init__(self,Weight=None,Moudle=None):
        if Weight !=None:
            self.weight=Weight
        elif Moudle !=None:
            self.weight=Moudle._parameters['weight'].T
        else:
            raise Exception('No Weight or Moudle')
        
    def __mul__(self, other):
        res = self.weight.matmul(other.weight)
        return NetMat(Weight=res)
    
    def __repr__(self) -> str:
        return f'NetMat(Weight={self.weight})'

x = torch.tensor([1,2,3]).float()
A=NetMat(Moudle=torch.nn.Linear(3, 5))
B=NetMat(Moudle=torch.nn.Linear(5, 5))
C=NetMat(Moudle=torch.nn.Linear(5, 1))

In [153]:
A*B*C

NetMat(Weight=tensor([[-0.0668],
        [ 0.0700],
        [ 0.0368]], grad_fn=<MmBackward0>))

In [154]:
x.matmul((A*B*C).weight)

tensor([0.1836], grad_fn=<SqueezeBackward4>)

网络运算符重载

In [1]:
from MoNet import *
layer()*layer()

MoNet(
  (Net): Sequential(
    (0): MoNet(
      (Net): Linear(in_features=10, out_features=1, bias=True)
    )
    (1): MoNet(
      (Net): Linear(in_features=10, out_features=1, bias=True)
    )
  )
)

In [2]:
Cell()

Cell(
  (Net): Sequential(
    (0:fc): Linear(in_features=10, out_features=1, bias=True)
    (1:bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2:act): PReLU(num_parameters=1)
    (3:dp): Dropout(p=0.5, inplace=False)
  )
)

In [3]:
import torch
from MoNet.monet import Cell, Layer

F=Layer(1,10)*Cell(10,10)*Layer(10,1)
print(F)
F(torch.tensor([[0.0],
                [1.0]]))

MoNet(
  (Net): Sequential(
    (0): MoNet(
      (Net): Sequential(
        (0): Layer(
          (Net): Linear(in_features=1, out_features=10, bias=True)
        )
        (1): Cell(
          (Net): Sequential(
            (0:fc): Linear(in_features=10, out_features=10, bias=True)
            (1:bn): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2:act): PReLU(num_parameters=1)
            (3:dp): Dropout(p=0.5, inplace=False)
          )
        )
      )
    )
    (1): Layer(
      (Net): Linear(in_features=10, out_features=1, bias=True)
    )
  )
)


tensor([[ 1.2049],
        [-0.3159]], grad_fn=<AddmmBackward0>)

In [4]:
from MoNet.monet import Mix

Mix()

Mix(
  (Net): Sequential(
    (0:input): Sequential(
      (cell-0): Sequential(
        (0:dp): Dropout(p=0.2, inplace=False)
      )
    )
    (1:hiddens): Sequential(
      (cell-0): Sequential(
        (0:fc): Linear(in_features=10, out_features=32, bias=True)
        (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2:act): PReLU(num_parameters=1)
        (3:dp): Dropout(p=0.5, inplace=False)
      )
      (cell-1): Sequential(
        (0:fc): Linear(in_features=32, out_features=32, bias=True)
        (1:bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2:act): PReLU(num_parameters=1)
        (3:dp): Dropout(p=0.5, inplace=False)
      )
    )
    (2:out): Sequential(
      (cell-0): Sequential(
        (0:fc): Linear(in_features=32, out_features=1, bias=True)
      )
    )
  )
)

In [5]:
Fc=Cell(1,5,['fc','act'])
Hid1=Cell(5,1,['fc','act'])
Hid2=Cell(5,1,['fc','act'])
Out = Layer(2,1,'fc')

x=torch.tensor([0.0])
F1=(Fc*Hid1+Fc*Hid2)*Out
F2=Fc*(Hid1+Hid2)*Out
F1(x)==F2(x)

tensor([True])